<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Brian Kim, Avishek Kumar, and Jonathan Morgan.

# Applications: Replication QWI Statistics with MO Wage Records
----

## Table of Contents

- [Introduction](#Introduction)
- [Basic Concepts](#Basic-Concepts)
- [Python Setup](#Python-Setup)
- [The Six Starter Files](#The-Six-Starter-Files)
- [Longitudinal Linkage](#Longitudinal-Linkage)
- [QWI Metrics](#QWI-Metrics)

## Introduction
- Back to [Table of Contents](#Table-of-Contents)

This notebook presents an application of creating variables, as taught in the "Variables" notebook. Here we will generate the relevant metrics of the Quarterly Workforce Indicators (QWI) framework, for the overall Missouri economy and by business EIN. 

QWI is a robust method of quantifying the dynamic employment patterns of workers and firms. It was developed by the Longitudinal Employer-Household Dynamics (LEHD) program at the Census in collaboration with state employment security agencies across the US. LEHD collects administrative unemployment insurance (UI) and employer (Quarterly Census of Employment and Wages) data from the state agencies. It then standardizes and cleans the data, applies some statistical imputation procedures, calculates statistics, and finally makes aggregates available to the public and microdata available to researchers. This procedure is described in detail in "The LEHD Infrastructure Files and the Creation of the Quarterly Workforce Indicators" (Abowd et al. 2009. in "Producer Dynamics: New Evidence from Micro Data").

In this notebook, we present code for turning simple Wage Records files into QWI files, as well as examples and suggestions for optimal use. 

> In the following steps, we will code the QWI metrics for a given quarter (2010 quarter 1). If you need these metrics for project work, the results for all quarters have been run and stored on the ADRF, in the `ada_18_uchi` schema.

## Basic Concepts
- Back to [Table of Contents](#Table-of-Contents)

The QWI framework is focused on **jobs**, which are the combination of **people** and **employers**. If John works at both McDonalds and Burger King, this will generate two completely separate jobs in the QWI data. If John quits his job at McDonalds, that job will no longer exist in the QWI data. If John quits both of his jobs, he will not be observed in the QWI data at all. The Wage Records files might contain "jobs" where no money changes hands in a given quarter. From the perspective of QWI, a job only exists if it involves wages greater than one dollar.

In the ADRF environment, **People** are identified by their (hashed) Social Security Number and (hashed) name. In a perfect world, tracking people over time would simply be a matter of tracking their unique SSN. However, SSNs can be corrupted by data entry errors or used by multiple people. For the time being, we do not address these issues. We only longitudinally link two jobs if they have identical SSNs.

Due to the fact that people who are unemployed or out of the labor force do not appear in QWI data, these data are not suitable for calculating population-wide employment rates of the kind produced by the Bureau of Labor Statistics. However, under the assumption that people who do not appear in the Wage Records data do not have jobs, QWI data can be used to calculate employment rates and other employment statistics for a subpopulation of people with known (hashed) SSNs. 

In this QWI routine, **Employers** are identified by their Employer Identification Number (EIN). Other methodologies use the Unemployment Insurance Account Number (UI Account Number).

Each QWI file is specific to a single **quarter**. It includes a job if and only if it existed in that quarter. It includes information from prior and later quarters, but this is only to help construct the employment dynamics. The QWI file for 2010 quarter 2 gives a complete picture of the employment dynamics for that quarter but an incomplete (and likely misleading) picture of the employment dynamics for 2010 quarter 1 (or any other quarter). The Wage Records data tell us whether a job existed in a given quarter, but without more information we cannot differentiate a job that lasted the entire quarter from one that only lasted a day. 

## Python Setup
- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# general use imports
import datetime
import glob
import inspect
import numpy
import os
import six
import warnings
import math
from itertools import izip

# pandas-related imports
import pandas as pd
import sqlalchemy

# CSV file reading-related imports
import csv

# database interaction imports
import sqlalchemy
import datetime

In [ ]:
# to create a connection to the database, we need to pass the name of the database and host of the database
connection_string = "postgresql://10.10.2.10/appliedda"
conn = sqlalchemy.create_engine(connection_string)

In [ ]:
## makeDataDict: A function to split a dataset into 6 quarterly datasets for use in QWI calculations
## Inputs: 
## -connection: Connection to the database
## -keyYr: The year to caculate QWI stats for
## -keyQ: The quarter to calculate QWI stats for
## Output:
## -res: A dictionary with keys m4, m3, m2, m1, t, and p1 containing subsets of the longDat that condition
## on keyRy/keyQ (for entry 't'), 4 lags ('m1'-'m4'), and one lead ('p1')
def makeDataDict(connection, keyYr, keyQ):
    keys = ['m4', 'm3', 'm2', 'm1', 't', 'p1']
    res = {}
    for i in range(0,6):
        ##Find the right year and quarter, with i=0 corresponding to the 4th lag   
        yr = int(keyYr - 1 + math.floor((keyQ+i-1)/4))
        q = int(keyQ + i - 4*math.floor((keyQ+i-1)/4))
        
        # Query the dataset on the given year and quarter, keeping only identifiers and wage as columns 
        query='''
        SELECT ssn
                , ein
                , 1 as wage 
        FROM il_des_kcmo.il_wage
        WHERE year = {} and quarter = {}
        '''.format(yr, q)
        
        res[keys[i]] = pd.read_sql(query, connection)
    
    return res

In [ ]:
## linkData: A function to longitudinally link the datasets supplied in the form of a dictionary output by 
## makedataDict. Currently links by a deterministic left merge (where 't' is always on the left) by SSN and EIN.
## Input:
## -dataDict: A dictionary with keys m4, m3, m2, m1, t, and p1 containing subsets of the longDat that condition
## on keyRy/keyQ (for entry 't'), 4 lags ('m1'-'m4'), and one lead ('p1')
## Output: 
## -res: A single dataframe with columns SSN, EIN, and wage_m4-wage_p1, the results of the 
## longitudinal linkage. A job is included iff it exists in period 't'. When that job does not exist in a different
## period, the record describing that job will have  a missing value in the column for that period. 
def linkData(dataDict):
    for time in dataDict:
        dataDict[time] = dataDict[time][(dataDict[time]['wage'].notnull())]
    res = dataDict['t']
    for time in dataDict:
        if time != 't':
            ##Define the suffix to add to the time period being merged in 
            suff = '_' + time
            ##Merge on iddssn, uiacctno. Keep all records in 't'. Drop unmatched records in the other period. 
            res = pd.merge(left=res, right=dataDict[time], on=['ssn','ein'], how='left', suffixes=('',suff))
    res = res[['ssn', 'ein', 'wage', 'wage_m4', 'wage_m3', 'wage_m2', 'wage_m1', 'wage_p1']]
    return res

In [ ]:
## makeStatistics: A function to calculate QWI statistics 
## (employment-stable employment, accessions and separations therefrom, etc.). 
## Input:
## -linkData: A single dataframe with columns idssn, uiacctno, and wage_m4-wage_p1. 
## Missing entries will be interpreted as the non-existence of a job.
## Output:
## -res: The input dataset with columns appended for each statistic.

def makeStatistics(linkData):
    res = linkData
    
    #Flow Employment
    res['qwmt'] = 1*(res['wage'] > 0)
    res['qwmtm4'] = 1*(res['wage_m4'] > 0)
    res['qwmtm3'] = 1*(res['wage_m3'] > 0)
    res['qwmtm2'] = 1*(res['wage_m2'] > 0)
    res['qwmtm1'] = 1*(res['wage_m1'] > 0)
    res['qwmtp1'] = 1*(res['wage_p1'] > 0)
            
    #Beginning of Quarter Employment
    res['qwbt'] = 1*((res['qwmtm1']==1) & (res['qwmt']==1))

    #End of Quarter Employment
    res['qwet'] = 1*((res['qwmt']==1) & (res['qwmtp1']==1))

    #Full Quarter Employment
    res['qwft'] = 1*((res['qwmtm1']==1) & (res['qwmt']==1) & (res['qwmtp1']==1))

    #Accessions
    res['qwat'] = 1*((res['qwmtm1']==0) & (res['qwmt']==1))
    
    #Accessions to Consecutive Quarter Status
    res['qwa2t'] = 1*((res['qwat']==1) & (res['qwmtp1']==1))

    #Accessions to Full Quarter Status
    res['qwa3t'] = 1*((res['qwmtm2']==0) & (res['qwmtm1']==1) & (res['qwmt']==1) 
                      & (res['qwmtp1']==1))

    #Separations
    res['qwst'] = 1*((res['qwmt']==1) & (res['qwmtp1']==0))
    
    #New Hires
    res['qwht'] = 1*((res['qwmtm4']==0) & (res['qwmtm3']==0) & (res['qwmtm2']==0) 
                     & (res['qwmtm1']==0) & (res['qwmt']==1))

    #Recalls
    res['qwrt'] = 1*((res['qwmtm1']==0) & (res['qwmt']==1) & (res['qwht']==0))

    
    ##Replace technical names by more explicit names
    names_tech = (['qwmt','qwmtm4','qwmtm3','qwmtm2','qwmtm1','qwmtp1','qwbt'
                   ,'qwet','qwft','qwat','qwa2t','qwa3t','qwst','qwht','qwrt'])
    names_def =(['emp_current_qrt','emp_4qtrs_ago','emp_3qtrs_ago','emp_2qtrs_ago'
                 ,'emp_prev_qtr', 'emp_next_qtr','emp_begin_qtr','emp_end_qtr'
                 ,'emp_full_qtr','accessions_current', 'accessions_consecutive_qtr'
                 ,'accessions_full_qtr','separations','new_hires','recalls'])
    rn_dict = dict(izip(names_tech, names_def))
    res = res.rename(index=str, columns=rn_dict)
    
    return res

In [ ]:
qwi_vars =(['emp_current_qrt','emp_4qtrs_ago','emp_3qtrs_ago','emp_2qtrs_ago','emp_prev_qtr'
             , 'emp_next_qtr','emp_begin_qtr','emp_end_qtr','emp_full_qtr','accessions_current'
             , 'accessions_consecutive_qtr','accessions_full_qtr','separations','new_hires','recalls'])

In [ ]:
for i in range(2012,2016):
    full_data = makeDataDict(conn, i, 1)
    print("Downloaded " + str(i) + " data")
    
    linked_data = linkData(full_data)
    print("Linked "+ str(i) + " data")
    
    qwi_stats = makeStatistics(linked_data)
    qwi_ein_stats = qwi_stats.groupby(['ein'])[qwi_vars].mean().reset_index()
    qwi_ein_count = qwi_stats.groupby(['ein']).size().reset_index()
    qwi_ein_count['nb_empl'] = qwi_ein_count[0]
    del qwi_ein_count[0]
    qwi_ein = pd.merge(qwi_ein_count, qwi_ein_stats, on = 'ein')
    
    table_name = 'qwi_ein_' + str(i) + "_1"
    
    conn = sqlalchemy.create_engine(connection_string)
    qwi_ein.to_sql(table_name, conn, schema = 'ada_18_uchi')
    print("Uploaded " + str(i) + " data")